<a href="https://colab.research.google.com/github/ppiont/tensor-flow-state/blob/master/2_data_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/gdrive", force_remount = True)

In [6]:
%cd "/gdrive/My Drive/tensor-flow-state/tensor-flow-state"

/gdrive/My Drive/tensor-flow-state/tensor-flow-state


In [0]:
data_dir = "data/"
sensor_list = ["RWS01_MONIBAS_0021hrl0414ra", "RWS01_MONIBAS_0021hrl0403ra", "RWS01_MONIBAS_0021hrl0409ra", "RWS01_MONIBAS_0021hrl0420ra", "RWS01_MONIBAS_0021hrl0426ra"]

In [0]:
import pandas as pd
import os

def join_sensors(sensor_list):
    combined_df = pd.DataFrame({"timestamp": pd.date_range(start = "2011-01-01", end = "2019-12-31", freq = "T")})
    combined_df.set_index("timestamp", drop = True, inplace = True)
    for i, sensor in enumerate(sensor_list):
        # only add speed limit on the final sensor
        if i == len(sensor_list) - 1:
            df = pd.read_csv(data_dir + sensor + ".csv", index_col = 0, parse_dates = True)
            combined_df = combined_df.join(df, how = "outer", rsuffix = '_' + sensor)
        else:
            df = pd.read_csv(data_dir + sensor + ".csv", usecols = [0, 1, 2], index_col = 0, parse_dates = True)
            combined_df = combined_df.join(df, how = "outer", rsuffix = "_" + sensor)
    combined_df.dropna(how = "all", axis = 0, inplace = True) # this works in all cases because speed_limit is never NA on a sensor df
    return combined_df

In [0]:
# Join sensors to one table
df = join_sensors(sensor_list)

# Rename and reorder columns
df.rename({"speed_RWS01_MONIBAS_0021hrl0403ra": "speed_-2", "speed_RWS01_MONIBAS_0021hrl0409ra": "speed_-1",\
           "speed_RWS01_MONIBAS_0021hrl0420ra": "speed_+1", "speed_RWS01_MONIBAS_0021hrl0426ra": "speed_+2",\
           "flow_RWS01_MONIBAS_0021hrl0403ra": "flow_-2", "flow_RWS01_MONIBAS_0021hrl0409ra": "flow_-1",\
           "flow_RWS01_MONIBAS_0021hrl0420ra": "flow_+1", "flow_RWS01_MONIBAS_0021hrl0426ra": "flow_+2"\
           }, axis = 1, inplace = True)
col_order = ["speed", "flow", "speed_-2", "speed_-1","speed_+1", "speed_+2", "flow_-2", "flow_-1", "flow_+1", "flow_+2", "speed_limit"]
df = df[col_order]

# Save table to csv
df.to_csv(data_dir + "combined_df.csv")